In [1]:
from zipfile import ZipFile
from io import BytesIO
import datetime as dt
import pandas as pd
import numpy as np
import requests

## SQL to Pandas Example

### This is just a quick demo of converting SQL(In this case HQL) code to Pandas
#### The example I use is Hortonworks Clickstream example here -> http://hortonworks.com/hadoop-tutorial/how-to-visualize-website-clickstream-data/#section_2 (There is no particular reason why I used this demo)
#### The relevant poritions to this demo is just the data files regusers.tsv/urlmap.tsv/0.tsv
#### As well as the SQL Code which we are converting -

In [2]:
sql = """
select to_date(o.ts) logdate, o.url, o.ip, o.city, upper(o.state) state,
o.country, p.category, CAST(datediff( from_unixtime( unix_timestamp() ),
from_unixtime( unix_timestamp(u.birth_dt, 'dd-MMM-yy'))) / 365  AS INT) age, u.gender_cd
from omniture o
inner join products p     
on o.url = p.url
left outer join users u
on o.swid = concat('{', u.swid , '}')"""

### 1) Download and Create DFs

In [3]:
#Download and load zip file in memory
zipurl = 'https://s3.amazonaws.com/hw-sandbox/tutorial8/RefineDemoData.zip'
r = requests.get(zipurl)
z = ZipFile(BytesIO(r.content))

In [4]:
#Regusers and urlmap are easily read by passing \t delimiter
dfusers = pd.read_csv(z.open('data/users.tsv.gz'), sep = '\t', compression='gzip')
dfproduct = pd.read_csv(z.open('data/products.tsv.gz'), sep = '\t', compression='gzip')

In [5]:
#Omniture data has no header and per the example we only use 53 cols
dfominture = pd.read_csv(z.open('data/Omniture.0.tsv.gz'), sep = '\t', compression='gzip', header=None)
dfominture = dfominture.iloc[:,0:53]

#Create dummy cols
omniturecols = ['col_' + str(i) for i in range(1,54)]

#Assign dummy columns to omniture dataframe
dfominture.columns = omniturecols

/home/neil/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (14,15,18,21,22,51,69,75,76,77,79,108,109,113,114,125,176) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#We only need certain columns so lets create a new omniture df witht he columns we want
dfviewomniture = dfominture[["col_2", "col_8", "col_13","col_14", "col_50", "col_51", "col_53"]]
#Next lets assign more meaning full name to those columns
dfviewomniture.columns = ["ts", "ip", "url", "swid", "city", "country", "state"]

### 2) Joining DFs
First join we will replicate is the inner join, which joins omniture data and products data

In [7]:
dfprodomni = dfviewomniture.merge(dfproduct, how = 'inner', on = 'url', suffixes=('_omni', '_prod'))

Next join is replicating the left join between users and omniture data

In [8]:
#First thing we need to do is create a column in users df that has the SWID column concatentated with { }
dfusers['swid'] = '{' + dfusers['SWID'] + '}' 

#Now we can join the 2
dffinaljoin = dfprodomni.merge(dfusers, how = 'left', on = 'swid', suffixes=('_omniprod', '_users'))

## 3) Creating Custom Columns
The select porition of the sql statement does a few things, such as converting timestamp column to a date column, convert state to all caps, as well as calculating age based on the birthdate and current date

In [9]:
# First lets convert the ts column to date
dffinaljoin['logdate'] = dffinaljoin['ts'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S').date())

In [10]:
# Next lets create the state column with all upper case
dffinaljoin['state'] = dffinaljoin['state'].apply(lambda x: x.upper()) #Note this will replace the current state col

In [11]:
# Finally lets calculate the age
# Here is the function we will use to calculate age
def getAge(x):
    if type(x) == type(np.NaN):
        return np.NaN
    else:
        return int((dt.datetime.now().date() -\
       dt.datetime.strptime(x,'%d-%b-%y').date()).days / 365)
    
# Now to pass the function and create the column
dffinaljoin['age'] = dffinaljoin['BIRTH_DT'].apply(getAge)

In [12]:
# We have final df setup
# Last thing we should do is create a dataframe wiht only the columns we want
sql_to_df = dffinaljoin[['logdate', 'url', 'ip', 'city', 'state', 'country', 'category', 'age', 'GENDER_CD']]

In [14]:
#Display first 5 lines
sql_to_df.head()

,logdate,url,ip,city,state,country,category,age,GENDER_CD
0,2012-03-15,http://www.acme.com/SH55126545/VD55170364,99.122.210.248,homestead,FL,usa,home&garden,NaN,NaN
1,2012-03-15,http://www.acme.com/SH55126545/VD55170364,129.119.158.240,dallas,TX,usa,home&garden,26.0,F
2,2012-03-15,http://www.acme.com/SH55126545/VD55170364,71.53.206.175,charlottesville,VA,usa,home&garden,25.0,F
3,2012-03-15,http://www.acme.com/SH55126545/VD55170364,74.240.132.6,slidell,LA,usa,home&garden,25.0,U
4,2012-03-15,http://www.acme.com/SH55126545/VD55170364,74.190.188.100,atlanta,GA,usa,home&garden,25.0,M


In [16]:
#Finally lets filter (SQL Where) on GENDER_CD where equals 'F'
sql_to_df[(sql_to_df.GENDER_CD == 'F')].head() 

,logdate,url,ip,city,state,country,category,age,GENDER_CD
1,2012-03-15,http://www.acme.com/SH55126545/VD55170364,129.119.158.240,dallas,TX,usa,home&garden,26.0,F
2,2012-03-15,http://www.acme.com/SH55126545/VD55170364,71.53.206.175,charlottesville,VA,usa,home&garden,25.0,F
5,2012-03-15,http://www.acme.com/SH55126545/VD55170364,108.18.57.30,alexandria,VA,usa,home&garden,29.0,F
6,2012-03-15,http://www.acme.com/SH55126545/VD55170364,76.89.18.233,bridgeport,WV,usa,home&garden,25.0,F
8,2012-03-15,http://www.acme.com/SH55126545/VD55170364,99.45.50.183,nashville,TN,usa,home&garden,24.0,F
